# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [35]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

#%matplotlib inline

In [36]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

This image is: <class 'numpy.ndarray'> with dimesions: (540, 960, 3)


**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [37]:
import numpy as np
import cv2
import math
import os
import sys

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    # defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    # defining a 3 channel or 1 channel color to fill the mask with
    # depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # filling pixels inside the polygon defined by "vertices"
    # with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #cv2.imshow("Region of Interest", mask)
    #cv2.waitKey(0)
    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_line(img, line, color=[255,0,255], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    cv2.line(img, line[0], line[1], color, thickness)

def hough(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(
                img,
                rho,
                theta,
                threshold,
                np.array([]),
                minLineLength=min_line_len,
                maxLineGap=max_line_gap)
    return lines

def weighted_img(img, initial_img, alpha=0.8, beta=1., gamma=0.0):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * alpha + img * beta + gamma
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, alpha, img, beta, gamma)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [38]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidYellowLeft.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve2.jpg']

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [39]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Auxilary functions

Auxilary functions to drive `process_image` procedure.


In [40]:
def line_equation(start, end):
    m = (end[1] - start[1]) / (end[0] - start[0])
    b = start[1] - m * start[0]
    return m, b

def intersection_point(line1, line2):
    x = (line2[1] - line1[1]) / (line1[0] - line2[0])
    y = x * line1[0] + line1[1]
    return (x, y)

def stabilize_line(sublines, ulimit, dlimit):
    uline = line_equation((0, ulimit), (1, ulimit))
    dline = line_equation((0, dlimit), (1, dlimit))
    uxes, dxes = [], []
    for start, end in sublines:
        line = line_equation(start, end)
        ux, _ = intersection_point(uline, line)
        dx, _ = intersection_point(dline, line)
        uxes.append(ux)
        dxes.append(dx)
    ux = np.int32(np.median(uxes))
    dx = np.int32(np.median(dxes))
    return ((ux, ulimit), (dx, dlimit))

def draw_stable_lines(img, lines, ulimit, dlimit):
     lefts, rights = split_lines(lines)
     left = stabilize_line(lefts, ulimit, dlimit)
     right = stabilize_line(rights, ulimit, dlimit)
     draw_line(img, left, thickness=10)
     draw_line(img, right, thickness=10)

## Process image procedure

Process of finding lanes on image consists of six major parts:

1. Preprocess image: 
  1. Convert image to grayscale
  2. Apply gaussian blur to grayscaled image
  3. Search for edges using Canny edge detector
2. Select only interesting regions within image. In fact, for finding lanes there are only two interesting regions and they are symmetric trapezoids.
3. Enhance edge quality. I use dilation and erose methods of OpenCV. It helps exibit lane lines better.
4. Execute Hough algorithm for searching lines on gotten image.
5. After Hough algorithm there are plenty of different lines along lanes. There are short and long lines with jittering angles. At this stage I approximate right and left lines taking intersections with up and down limit lines (boundaries for lanes on image). I take robust statistic measure, median, on X axis for each intersection point. Hence after that computation I have 2 points for left and right lanes.
6. Last stage is simply merge operation of source image with final image.

In [41]:
def process_image(image, kernel_size=5, low_thr=50, high_thr=150):
    """
    `kernel_size` should be odd number 3, 5, 7 and etc.
    `low_thr` is lower threshold bound, which is used by
              Canny edge detector
    `high_thr` is high threshold bound, which is used by
              Canny edge detector
    `
    """

    # Part 1. Convert image to grayscale, blurring it and 
    #         searching edges using Canny algorithm with
    #         specified parameters
    im = grayscale(image)
    im = gaussian_blur(im, kernel_size)
    im = canny(im, low_thr, high_thr)
    
    # Part 2. Cut only interesting regions within image 
    #
    ylen, xlen, _ = image.shape
    xm, x8th, x20th = xlen // 2, xlen // 8, xlen // 25
    xl, xr = xm - x8th, xm + x8th
    ym = ylen // 2 + ylen // 10
    vertices = np.array([
            # Left polygon
            [(0, ylen),
             (xm - x20th, ym),
             (xm, ym),
             (xl, ylen)],
            # Right polygon
            [(xr, ylen),
             (xm, ym),
             (xm + x20th, ym),
             (xlen, ylen)]
        ], dtype=np.int32)
    
    im = region_of_interest(im, vertices)
    
    # Part 3. Enhance edge quality
    #
    elem = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    im = cv2.dilate(im, elem, iterations=3)
    im = cv2.erode(im, elem, iterations=2)
    
    # Part 4. Apply Hough algorithm for finding lines. Then draw lines
    #         on color image.
    rho = 1
    theta = np.pi/180
    threshold = 100
    min_len = 40
    max_gap = 50
    lines = hough(im, rho, theta, threshold, min_len, max_gap)
    
    im_with_lines = np.zeros((*im.shape, 3), dtype=np.uint8)
    
    # Part 5. Choose best right and left lane lines and 
    #         draw them on image.
    draw_stable_lines(im_with_lines, lines, ylen, ym + 20)

    # Part 6. Mix source image with drawn lines.
    # 
    final_image = weighted_img(im_with_lines, image, 1.0, 0.9)
    return final_image


Let's try the one with the solid white lane on the right first ...

In [42]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:05<00:00, 37.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 13.2 s, sys: 104 ms, total: 13.4 s
Wall time: 6.35 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [43]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [44]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:20<00:00, 33.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 41.3 s, sys: 284 ms, total: 41.6 s
Wall time: 20.8 s


In [31]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.

**NOTE: I'm not fully satisfied with result because of jittering of lines in videos. But I guess it can be solved by more efficient algorithm for getting skeleton. The wide tails of lanes produce a lot lines with different angle. I compute median of x coordiates of intersections with bommom and top y limits, it helps to stabilize the line, but not so much as I expected.**


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [32]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:12<00:00, 20.26it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 19.2 s, sys: 292 ms, total: 19.5 s
Wall time: 13.3 s


In [33]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))